In [ ]:
from misc import load, load_class_labels
import handcrafted as hc
import bag_of_words as bow

dataset_root = "./dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 0.5
train_info = load(dataset_root + 'train_info_dirty.csv', 1, sample_rate)
val_info = load(dataset_root + 'val_info.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

In [ ]:
from multiprocessing.pool import ThreadPool

pool = ThreadPool(processes=2)

features = pool.apply_async(hc.extract_features, (train_dir, train_image_names,
                                                  [(lambda img: hc.lbp(img, distances=[1, 2]))]))
handcrafted = pool.apply_async(hc.extract_features, (train_dir, train_image_names,
                                                     [(lambda img: hc.sift(img))]))

handcrafted = handcrafted.get()

In [ ]:
from misc import unroll_arrays
prob = 0.15
voc_size = 300

unrolled = unroll_arrays(handcrafted, prob)
kmeans = bow.fit(unrolled, vocabulary_size=voc_size, verbose=True, n_init=1, max_iter=50)

In [ ]:
from bag_of_words import predict
from misc import center_scale_columns
import numpy as np

bag_of_words = predict(kmeans, handcrafted)

features = features.get()
composed = np.hstack((bag_of_words, features))
features_cs = center_scale_columns(composed, train_labels)

In [ ]:
from misc import group_stats
feature_mean, feature_cov = group_stats(composed, train_labels, numeric_correction=1e-15)

In [ ]:
from image_discard import find_outliers_iter
names, distances = find_outliers_iter(composed, train_labels, train_image_names, threshold=15.5, num_iter=50)

In [ ]:
from image_discard import write_discarded_images
write_discarded_images(names, class_labels, "./discarded/", train_dir, True)